In [7]:
import importlib
import hashlib
from pathlib import Path
from benchmark_utils import render_distribution, visual_benchmark

# Directory with implementations
IMPLS_DIR = Path("impls")

# Cache: module_name -> {"hash": sha1, "result": benchmark_result}
benchmark_cache = {}

In [8]:
# Just restart the notebook and it will run all benchmarks again.
# Cached benchmarks are stored in memory and reused if the source file did not change.


def compute_sha1(path):
    """Return SHA1 hash of the file's contents."""
    with open(path, "rb") as f:
        return hashlib.sha1(f.read()).hexdigest()


def load_module(name):
    """Import or reload impls.<name>."""
    full_name = "impls." + name

    if full_name in globals():
        module = importlib.reload(globals()[full_name])
    else:
        module = importlib.import_module(full_name)
        globals()[full_name] = module

    return module


def report(stats):
    render_distribution(stats)
    print(f"Expected time to recover: {stats['expected_time']} packets")
    print(f"Packet efficency: {stats['packet_efficiency']} payload bits/packet")
    print(f"Bit Efficency: {stats['bit_efficency']} payload bits/bit")


# Iterate over implementation files
for file in IMPLS_DIR.glob("*.py"):
    name = file.stem

    # Skip interface and private modules
    if name == "_interface" or name.startswith("_"):
        continue

    print("Found implementation:", name)

    file_hash = compute_sha1(file)

    # If cached: check whether file changed
    if name in benchmark_cache:
        cached = benchmark_cache[name]
        cached_hash = cached["hash"]
        cached_stats = cached["stats"]

        if cached_hash == file_hash:
            print("  No changes. Using cached benchmark.")
            report(cached_stats)
            continue
        else:
            print("  File changed. Reloading and benchmarking again.")
    else:
        print("  First time seen. Benchmarking.")

    # Load/reload implementation
    module = load_module(name)

    # Require expected constructors
    if not hasattr(module, "producer_constructor") or not hasattr(
        module, "recoverer_constructor"
    ):
        print("  Missing constructors. Skipping.\n")
        continue

    N = 5
    D = 2**20
    producer_constructor = module.producer_constructor
    recoverer_constructor = module.recoverer_constructor
    D = getattr(module, "override_D", lambda _: D)(N)

    # Run benchmark
    stats = visual_benchmark(
        producer_constructor,
        recoverer_constructor,
        N=N,
        D=D,
        passes=10000,
    )

    # Store into cache
    benchmark_cache[name] = {
        "hash": file_hash,
        "stats": stats,
    }

    # Render the new result
    report(stats)

Found implementation: single_binary_fullrank
  First time seen. Benchmarking.


100%|██████████| 10000/10000 [00:12<00:00, 779.76it/s]


0 failures.


Expected time to recover: 25.515900000000002 packets
Packet efficency: 0.9113299679327067 payload bits/packet
Bit Efficency: 0.18226599358654133 payload bits/bit
Found implementation: single_binary_matrix
  First time seen. Benchmarking.


100%|██████████| 10000/10000 [00:21<00:00, 459.09it/s]
c:\Users\shich\Src\bottleneck_transmission\src\benchmark_utils.py:141: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



0 failures.


Expected time to recover: 70.0703 packets
Packet efficency: 0.3567845435227193 payload bits/packet
Bit Efficency: 0.07135690870454386 payload bits/bit
Found implementation: single_primitive_binary_matrix
  First time seen. Benchmarking.


100%|██████████| 10000/10000 [00:02<00:00, 3460.33it/s]

0 failures.


Expected time to recover: 9.1831 packets
Packet efficency: 0.2814912720890719 payload bits/packet
Bit Efficency: 0.056298254417814386 payload bits/bit
